Document showing how to consolidate the possible subsegments into a major trend. 

# Dependencies

In [1]:
import os
import sys

sys.path

['/Users/ernesto/PycharmProjects/FOREX/src',
 '/Users/ernesto/miniconda3/envs/python3_5_env/lib/python35.zip',
 '/Users/ernesto/miniconda3/envs/python3_5_env/lib/python3.5',
 '/Users/ernesto/miniconda3/envs/python3_5_env/lib/python3.5/plat-darwin',
 '/Users/ernesto/miniconda3/envs/python3_5_env/lib/python3.5/lib-dynload',
 '',
 '/Users/ernesto/miniconda3/envs/python3_5_env/lib/python3.5/site-packages',
 '/Users/ernesto/miniconda3/envs/python3_5_env/lib/python3.5/site-packages/IPython/extensions',
 '/Users/ernesto/.ipython']

In [2]:
import oanda_api

# Test case

Here I am going to use the uptrend shown below:
that goes from EUR_NZD (D) 26MAR2019 to 18JUN2019
![alt text](img/major_trend_eurnzd.png)

Let's get a CandleList for this uptrend

In [ ]:
oanda = OandaAPI(url='https://api-fxtrade.oanda.com/v1/candles?',
                     instrument='AUD_USD',
                     granularity='D',
                     alignmentTimezone='Europe/London',
                     dailyAlignment=22)

oanda.run(start='2019-03-08T22:00:00',
              end='2019-08-09T22:00:00',
              roll=True)

candle_list = oanda.fetch_candleset()

cl = CandleList(candle_list, instrument='AUD_USD', type='long')